In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.io as sio
import os

from multiprocessing import Pool
from tqdm import tqdm
from teaspoon.parameter_selection.FNN_n import FNN_n
from teaspoon.parameter_selection.MI_delay import MI_for_delay

from functions import init_Frame, SSub_process, tuplinator

from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
# Conditions codes
conditions_dict = {'con_left_self': 'S__1',
                   'con_left_other': 'S__2',
                   'con_right_self': 'S__3',
                   'con_right_other': 'S__4',
                   'uncon_left_self': 'S_11',
                   'uncon_left_other': 'S_12',
                   'uncon_right_self': 'S_13',
                   'uncon_right_other': 'S_14',}
cond_group = ['con_left_self',]

sub_list = ['001','002','003','004','005','006','007','008','009','010',
            '011','012','013','014','015','016','017','018','019','020',
            '023','024','025','026','028','029','030',
            '031','033','034','035','036','037','038','040',
            '042']

In [ ]:
# Load channel names
path = '/home/lunis/Documents/EEG/data/backward_masking/subj001_band_resample/channel.mat'
ch_clust = ['Fp1', 'Fp2', 'F3', 'F4', 'F7', 'F8', 'T7', 'T8', 'FT7',
             'FT8', 'O1', 'O2', 'PO7', 'PO8', 'P5', 'P6', 'P7', 'P8']
#ch_clust = ['P5', 'P7', 'PO8', 'P6', 'P8']
#ch_clust = ['P5', 'P7', 'P6'], ['P6', 'P8']
#ch_clust = ['P5']

ch_clust = tuplinator(ch_clust)

metrics = ['t', 'n']

conditions = ['S__', 'S_1']


# Load the .mat file (adjust the path)
mat_data = sio.loadmat(path)

ch_list = []
for m in mat_data['Channel'][0]:
#     print(m[0][0])
    ch_list.append(str(m[0][0]))

if type(ch_clust) ==  tuple:

    first = True
    for c in ch_clust:
        part = []
        
        for sc in c:
        
            part = part + [np.where(np.asarray(ch_list)==sc)[0][0]]

        print(part)
        part = part,
        if first == True:

            ch_clust_idx = part
            first = False

        else:
            ch_clust_idx = ch_clust_idx + part
            

else:

    ch_clust_idx = []
    for c in ch_clust:
        ch_clust_idx.append(np.where(np.asarray(ch_list)==c)[0][0])

df = init_Frame(conditions, ch_clust_idx, metrics)
df

In [7]:
#Set multiprocessing parameters
workers = os.cpu_count()-4
chk_size = int(len(sub_list)/workers) + 1

#Create iterable function
def Sub_Results(subID):

    rs = SSub_process(subID, conds = conditions, channels_idx = ch_clust_idx, DataFrame = df)

    return rs

#Run calculations
if __name__ == '__main__':
    with Pool(workers) as p:
        rows = p.imap(Sub_Results, sub_list, chk_size)

        i = 0
        for row in rows:

            df.loc[i] = row

            i = i +1

#Save results
df.to_csv('embedding.csv', index = False)
df.head()

029: Done!
035: Done!
001: Done!
011: Done!
023: Done!
042: Done!
006: Done!
016: Done!
030: Done!
002: Done!
036: Done!
024: Done!
012: Done!
007: Done!
025: Done!
031: Done!
017: Done!
003: Done!
013: Done!
037: Done!
026: Done!
008: Done!
033: Done!
004: Done!
028: Done!
038: Done!
018: Done!
014: Done!
009: Done!
034: Done!
005: Done!


/tmp/ipykernel_70800/3683904713.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i] = row


040: Done!
019: Done!
015: Done!
010: Done!
020: Done!


,SubID,t S__ [0],n S__ [0],t S__ [1],n S__ [1],t S__ [2],n S__ [2],t S__ [3],n S__ [3],t S__ [10],...,t S_1 [57],n S_1 [57],t S_1 [48],n S_1 [48],t S_1 [49],n S_1 [49],t S_1 [14],n S_1 [14],t S_1 [15],n S_1 [15]
0,001,13,3,16,3,17,3,15,3,13,...,17,3,10,3,23,3,15,3,19,3
1,002,8,3,13,3,10,3,10,3,7,...,19,3,9,3,7,3,7,3,10,3
2,003,10,3,11,3,14,3,13,3,13,...,16,3,14,3,13,3,14,3,15,3
3,004,16,3,15,3,21,3,17,3,11,...,19,3,8,3,14,3,19,3,15,3
4,005,17,3,12,3,15,3,14,3,10,...,14,3,15,2,15,3,11,3,18,3


In [ ]:
df = pd.read_csv('embedding.csv')
print('Average conscious embedding: ' +  str(df['S__ n'].mean()) + ' +- ' + str(np.sqrt(df['S__ n'].var())))
print('Average unconscious embedding: ' +  str(df['S_1 n'].mean()) + ' +- ' + str(np.sqrt(df['S_1 n'].var())))

fig,axs = plt.subplots(1,1, figsize=(13,5))

axs = plt.hist(df['S__ tau'], 20, range = (5, 30), alpha = 0.5)
axs = plt.hist(df['S_1 tau'], 20, range = (5, 30), alpha = 0.5)

plt.show()


In [ ]:
conds = ['S__', 'S_1'] # S__ : conscious, S_1 : unconscious
fig,axs = plt.subplots(6,6, figsize=(12,10))
for i, ax in enumerate(axs.flat):
    print('sub '+str(i))
    sub = sub_list[i]
    folder = '/home/lunis/Documents/EEG/data/backward_masking/subj' + sub + '_band_resample/'
    all_files = os.listdir(folder)
    # cond = 'S_1' # unconscious

    # Loop over conditions (conscious and unconscious)
    for cond in conds:
        my_cond_files = [f for f in all_files if cond in f ]

        # Loop over trials of the present conditions
        all_trials = np.empty((0,451))
        for f in my_cond_files:
            path = folder+f
            # Load the .mat file (adjust the path)
            mat_data = sio.loadmat(path)

            # Inspect the keys in the .mat file
    #         print(mat_data['F'][ch_clust_idx].shape)
            data = mat_data['F'][ch_clust_idx]
    #         plt.plot(data.mean(axis=0))
        #     for e in data:
        #         plt.plot(e)
            all_trials = np.concatenate((all_trials, data.mean(axis=0)[np.newaxis, :]))
#         print(all_trials.shape)
        ax.plot(all_trials.mean(axis=0), label=cond)
        ax.axvspan(150, 250, color='grey', alpha=0.15)
#     ax.legend()
plt.savefig('plottini_all_sub_18_el.png', dpi=300)